# Data Indexing and Selection

## Seleção de Dados em Series

Como vimos na seção anterior, um objeto ``Series`` age de muitas maneiras como um array NumPy unidimensional e, em muitos aspectos, como um dicionário Python padrão.
Se mantivermos essas duas analogias sobrepostas em mente, isso nos ajudará a entender os padrões de indexação e seleção de dados nesses arrays.

### Series como dicionário

Como um dicionário, o objeto ``Series`` fornece um mapeamento de uma coleção de chaves para uma coleção de valores:

In [ ]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [ ]:
data['b']

0.5

Também podemos usar expressões e métodos semelhantes a dicionários em Python para examinar as chaves/índices e valores:

In [ ]:
'a' in data

True

In [ ]:
data.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [ ]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

Os objetos ``Series`` podem até ser modificados com uma sintaxe semelhante a um dicionário.

Da mesma forma que você pode aumentar um dicionário fazendo uma atribuição a uma nova chave, você pode aumentar uma ``Series`` fazendo uma nova atribuição a um índice:

In [ ]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

Essa mutabilidade fácil dos objetos é um recurso conveniente: nos bastidores, o Pandas está tomando decisões sobre o layout da memória e a cópia de dados que podem ser necessárias; o usuário geralmente não precisa se preocupar com essas questões.

### Series como um array unidimensional



Uma ``Series`` se baseia nessa interface semelhante a um dicionário e fornece seleção de itens no estilo de array pelos mesmos mecanismos básicos dos arrays NumPy - ou seja, *fatias*, *máscaras* e *indexação avançada*.
Exemplos desses são os seguintes:

In [ ]:
# slicing pelo índice explícito
data['a':'d']

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [ ]:
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

In [ ]:
# slicing pelo índice inteiro implícito
data[2:4]

c    0.75
d    1.00
dtype: float64

In [ ]:
# masking
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [ ]:
# fancy indexing
data[['a', 'e']]

a    0.25
e    1.25
dtype: float64

Entre esses, o fatiamento pode ser a fonte da maior confusão.

Observe que ao fatiar com um índice explícito (ou seja, ``data['a':'c']``), o índice final é *incluído* na fatia, enquanto ao fatiar com um índice implícito (ou seja, ``data[0:2]``), o índice final é *excluído* da fatia.

### Indexadores: loc, iloc

Essas convenções de fatiamento e indexação podem ser uma fonte de confusão.

Por exemplo, se sua ``Series`` tiver um índice inteiro explícito, uma operação de indexação como ``data[1]`` usará os índices explícitos, enquanto uma operação de fatiamento como ``data[1:3]`` usará o índice no estilo Python implícito.

In [ ]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

In [ ]:
# indice explícito quando faz indexação
data[3]

In [ ]:
# índice implícito quando faz slicing
data[1:3]

Por causa dessa potencial confusão no caso de índices inteiros, o Pandas fornece alguns atributos de indexação especiais que expõem explicitamente certos esquemas de indexação.
Esses não são métodos funcionais, mas atributos que expõem uma interface de fatiamento particular para os dados na Series.

Primeiro, o atributo loc permite a indexação e o fatiamento que sempre fazem referência ao índice explícito:

In [ ]:
data.loc[1]

In [ ]:
data.loc[1:3]

O atributo ``iloc`` permite a indexação e o fatiamento que sempre fazem referência ao índice no estilo Python implícito:

In [ ]:
data.iloc[1]

In [ ]:
data.iloc[1:3]

Um princípio orientador do código Python é que "o explícito é melhor que o implícito."
A natureza explícita de ``loc`` e ``iloc`` os torna muito úteis na manutenção de um código limpo e legível; especialmente no caso de índices inteiros, recomendo usar ambos para tornar o código mais fácil de ler e entender, e para evitar bugs sutis devido à convenção mista de indexação/fatiamento.

## Seleção de Dados em DataFrame

Lembre-se de que um ``DataFrame`` age de muitas maneiras como um array bidimensional ou estruturado, e de outras maneiras como um dicionário de estruturas ``Series`` que compartilham o mesmo índice.
Essas analogias podem ser úteis para ter em mente enquanto exploramos a seleção de dados dentro dessa estrutura.

### DataFrame como um dicionário

A primeira analogia que consideraremos é o ``DataFrame`` como um dicionário de objetos ``Series`` relacionados.
Vamos retornar ao nosso exemplo de áreas e populações dos estados:

In [ ]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop})
data

As ``Series`` individuais que compõem as colunas do ``DataFrame`` podem ser acessadas através da indexação no estilo de dicionário do nome da coluna:

In [ ]:
data['area']

Equivalentemente, podemos usar o acesso no estilo de atributo com nomes de colunas que são strings:

In [ ]:
data.area

Podemos confirar que as duas formas acessam o mesmo objeto:

In [ ]:
data.area is data['area']

Embora isso seja uma abreviação útil, lembre-se de que não funciona em todos os casos!

Por exemplo, se os nomes das colunas não forem strings, ou se os nomes das colunas entrarem em conflito com os métodos do ``DataFrame``, esse acesso no estilo de atributo não será possível.

Por exemplo, o ``DataFrame`` tem um método ``pop()``, então ``data.pop`` apontará para isso em vez da coluna ``"pop"``:

In [ ]:
data.pop is data['pop']

Em particular, você deve evitar a tentação de tentar atribuir colunas via atributo (ou seja, use ``data['pop'] = z`` em vez de ``data.pop = z``).

Assim como com os objetos ``Series`` discutidos anteriormente, essa sintaxe no estilo de dicionário também pode ser usada para modificar o objeto, neste caso, adicionando uma nova coluna:

In [ ]:
data['density'] = data['pop'] / data['area']
data

### DataFrame como array bidimensional

Como mencionado anteriormente, também podemos ver o ``DataFrame`` como um array bidimensional aprimorado.
Podemos examinar o array de dados subjacente bruto usando o atributo ``values``:

In [ ]:
data

In [ ]:
data.values

Com isso em mente, muitas observações familiares semelhantes a arrays podem ser feitas no próprio ``DataFrame``.

Por exemplo, podemos transpor o ``DataFrame`` completo para trocar linhas e colunas:

In [ ]:
data.T

Quando se trata de indexação de objetos ``DataFrame``, no entanto, é claro que a indexação no estilo de dicionário das colunas impede nossa capacidade de simplesmente tratá-lo como um array NumPy.
Em particular, ao passar um único índice para um array vamos acessar uma linha:

In [ ]:
data.values[0]

e passar um único "índice" para um ``DataFrame`` acessa uma coluna:

In [ ]:
data['area']

Portanto, para indexação no estilo de array, precisamos de outra convenção.
Aqui, o Pandas novamente usa os indexadores ``loc``, ``iloc`` e ``ix`` mencionados anteriormente.

Usando o indexador ``iloc``, podemos indexar o array subjacente como se fosse um simples array NumPy (usando o índice no estilo Python implícito), mas os rótulos de índice e coluna do ``DataFrame`` são mantidos no resultado:

In [ ]:
data.iloc[:3, :2]

Da mesma forma, usando o indexador ``loc``, podemos indexar os dados subjacentes em um estilo semelhante a um array, mas usando os nomes explícitos de índice e coluna:

In [ ]:
data.loc[:'Illinois', :'pop']

Qualquer um dos padrões familiares de acesso a dados no estilo NumPy pode ser usado dentro desses indexadores. Por exemplo, no indexador loc, podemos combinar máscaras e indexação sofisticada como no seguinte:

In [ ]:
data.loc[data.density > 100, ['pop', 'density']]

Qualquer uma dessas convenções de indexação também pode ser usada para definir ou modificar valores; isso é feito da maneira padrão como com o NumPy:

In [ ]:
data.iloc[0, 2] = 90
data

### Convenções de indexação adicionais

Existem algumas convenções de indexação extras que podem parecer em desacordo com a discussão anterior, mas, mesmo assim, podem ser muito úteis na prática.
Primeiro, enquanto a *indexação* se refere a colunas, o *fatiamento* se refere a linhas:

In [ ]:
data['Florida':'Illinois']

Essas fatias também podem se referir a linhas por número, em vez de por índice:

In [ ]:
data[1:3]

Da mesma forma, as operações de mascaramento direto também são interpretadas linha a linha, em vez de coluna a coluna:

In [ ]:
data[data.density > 100]